# FrostyFriday Week17 ⛄

First, check dataset "OPENSTREETMAP_NEW_YORK.NEW_YORK" by SQL.  

In [ ]:
SELECT ID, COORDINATES, ADDR_CITY FROM OPENSTREETMAP_NEW_YORK.NEW_YORK.v_osm_ny_shop_electronics LIMIT 100;

Next, test basic processing for this dataset.  
Coordinates includes various shapes, so I used ST_CENTROID function and standarized these.  

In [ ]:
SELECT id, ST_CENTROID(coordinates) AS point, addr_city FROM OPENSTREETMAP_NEW_YORK.NEW_YORK.v_osm_ny_shop_electronics WHERE addr_city IS NOT NULL;

## Pairing step  

Create pair of nodes. Node means POI(Point of Interest) like a shop.  
I defined some conditions:  

- ID of nodes must be different.
- Reference node is in Brooklyn.
- ID which is related to 3 or more pairs are extracted by QUALIFY.

In [ ]:
SELECT p.id,
       q.id AS neighbor_id,
       ST_CENTROID(p.coordinates) as point1,
       ST_CENTROID(q.coordinates) as point2,
       ST_DISTANCE(point1, point2) AS distance,
       ARRAY_MIN([ST_X(point1), ST_X(point2)]) AS min_x,
       ARRAY_MAX([ST_X(point1), ST_X(point2)]) AS max_x,
       ARRAY_MIN([ST_Y(point1), ST_Y(point2)]) AS min_y,
       ARRAY_MAX([ST_Y(point1), ST_Y(point2)]) AS max_y,
       COUNT(*) OVER (PARTITION BY p.id) poi_count
FROM OPENSTREETMAP_NEW_YORK.NEW_YORK.v_osm_ny_shop_electronics p
JOIN OPENSTREETMAP_NEW_YORK.NEW_YORK.v_osm_ny_shop_electronics q
ON p.addr_city = 'Brooklyn'
   AND p.id != q.id
   AND distance <= 750
QUALIFY poi_count >= 3;

## Marge step

Marge pairs.  
I didn't use GEOGRAPHY data in the above cell, because these are changed while referenced from a cell to another cell. (Behavior as of 8/3/2024)

In [ ]:
SELECT g.id,
MIN(g.min_x) AS min_x,
MAX(g.max_x) AS max_x,
MIN(g.min_y) AS min_y,
MAX(g.max_y) AS max_y
FROM {{cells.cell4}} g
GROUP BY g.id;

## Polygonize step

Create polygons from min and max values.  
I used strings concat approach to create polygons.  
X means longitude, Y means latitude. The order is clockwise.  

In [ ]:
select id, 
   TO_GEOGRAPHY('POLYGON((' ||
        min_x || ' ' || min_y || ', ' ||
        min_x || ' ' || max_y || ', ' ||
        max_x || ' ' || max_y || ', ' ||
        max_x || ' ' || min_y || ', ' ||
        min_x || ' ' || min_y || 
    '))') AS bounding_box,
    ST_ASWKT(bounding_box) as wkt
from {{cells.cell5}};

## Answer1

I used ST_COLLECT functions to marge an array of polygons to one multi-polygon.  
I used WKT->GEOGRAPHY->WKT conversion approach because of the previously mentioned behavior.  


In [ ]:
SELECT ST_ASWKT(ST_COLLECT(ST_GEOGRAPHYFROMWKT(wkt))) AS wkts
FROM {{cells.cell6}};

I tried another visualize approach.  
To visualize by using Streamlit, first create an array of WKTs.  

In [ ]:
import streamlit as st

df = cell6.to_pandas()

wkts = []
for _, row in df.iterrows():
    wkts.append(row["WKT"])
st.write(wkts)

Snowflake Notebooks can visualize geo data by using Streamlit, GeoPandas and PyDeck!  

In [ ]:
import geopandas
import pydeck

# GeoPandasが扱えるデータ型に変換する
geodata = geopandas.GeoSeries.from_wkt(wkts)
st.write(geodata)

# 地図レイヤーの作成
layer = pydeck.Layer(
    type="GeoJsonLayer",
    data=geodata,
    get_fill_color=[180, 0, 200, 140]
)

# 地図の初期ビュー
INITIAL_VIEW_STATE = pydeck.ViewState(
    latitude=40.700,
    longitude=-73.900,
    zoom=11,
    max_zoom=16
)

# 地図を表示
fig = pydeck.Deck(
    layers=[layer],
    map_style="mapbox://styles/mapbox/light-v9",
    initial_view_state=INITIAL_VIEW_STATE
)
st.pydeck_chart(fig)